In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn.metrics import f1_score, precision_score, recall_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv("/content/drive/MyDrive/train_top_10_pp.csv")
df.head()

,Unnamed: 0,Title,Tags,tag_count,description,code
0,0,how check uploaded file image without mime type,php,1,id like check uploaded file image file eg png ...,NaN
1,1,how i replace special characters url,c#,1,this probably simple i simply cannot find answ...,NaN
2,2,how fetch xml feed using aspnet,c# asp.net,2,ive decided convert windows phone app fetches ...,NaN
3,3,php framework url conventions,php,1,a lot frameworks use url conventions like grea...,/controller/action/{id}
4,4,play framework auto javascript css minifier,javascript,1,does anyone know good play plugin automaticall...,NaN


In [4]:
vectorizer = CountVectorizer(tokenizer = lambda x: x.split())
tag_bow = vectorizer.fit_transform(df['Tags'])

In [5]:
tags = vectorizer.get_feature_names()
tags

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['android',
 'asp.net',
 'c#',
 'c++',
 'iphone',
 'java',
 'javascript',
 'jquery',
 'php',
 'python']

In [6]:
tags[3] = "c+"
tags

['android',
 'asp.net',
 'c#',
 'c+',
 'iphone',
 'java',
 'javascript',
 'jquery',
 'php',
 'python']

In [7]:
new_df = df.loc[df['Tags'].str.contains("c+")]
len(new_df)

722915

In [8]:
new_df = df.loc[df['Tags'].str.contains('android', case=False)]
print(len(new_df))
dataset=new_df[:6000]
#print(len(dataset))
for tag in tags:
  new_df = df.loc[df['Tags'].str.contains(tag)]
  new_df = new_df[:6000]
  print(len(new_df))
  dataset = dataset.append(new_df)
  #duplicate_pairs = dataset.sort_values('Title', ascending=False).duplicated('Title')
  #print("Total number of duplicate questions : ", duplicate_pairs.sum())
  #dataset = dataset[~ duplicate_pairs]
  dataset = dataset.drop_duplicates(keep='first')
  print(tag," ",len(dataset))

232412
6000
android   6000
6000
asp.net   11996
6000
c#   17156
6000
c+   20389
6000
iphone   26318
6000
java   29645
6000
javascript   32572
6000
jquery   36408
6000
php   41686
6000
python   47559


In [9]:
dataset.shape

(47559, 6)

In [10]:
dataset.head()

,Unnamed: 0,Title,Tags,tag_count,description,code
13,13,loadable kernel module notification manager,android,1,ive successfully created installed loadable ke...,NaN
45,45,eclipse indigo allow override non activity ove...,android,1,i using eclipse indigo android development the...,NaN
52,52,android v webview loaddatawithbaseurl load page,android,1,when im loading page using system load page on...,.loadData
54,54,android changing locale within app affect firs...,java android,2,i use code bellow setcontentview main activity...,"Locale locale = new Locale(""ar"");"
58,58,android client socket unknowhostexception,java android,2,i working server application java multithreade...,NaN


In [11]:
vectorizer = CountVectorizer(tokenizer = lambda x: x.split(), binary='true')
y_multilabel = vectorizer.fit_transform(dataset['Tags'])

In [12]:
X_train, X_test, y_train, y_test = train_test_split(dataset, y_multilabel, test_size = 0.2, random_state = 42)

print("Number of data points in training data :", X_train.shape[0])
print("Number of data points in test data :", X_test.shape[0])

Number of data points in training data : 38047
Number of data points in test data : 9512


In [13]:
vectorizer = TfidfVectorizer(tokenizer = lambda x: x.split())

X_train_multilabel = vectorizer.fit_transform(X_train["Title"])
X_test_multilabel = vectorizer.transform(X_test["Title"])

In [14]:
print("Training data shape X : ", X_train_multilabel.shape, "Y :", y_train.shape)
print("Test data shape X : ", X_test_multilabel.shape, "Y:", y_test.shape)

Training data shape X :  (38047, 19168) Y : (38047, 10)
Test data shape X :  (9512, 19168) Y: (9512, 10)


In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier

sgd = SGDClassifier(loss='hinge',alpha=0.01)
lr = LogisticRegression(penalty='l2',tol=1e-4)
svc = LinearSVC(loss='squared_hinge',C=1.0)
knn = KNeighborsClassifier(n_neighbors=5,metric='minkowski')

for classifier in [sgd, lr, svc, knn]:
    clf = OneVsRestClassifier(classifier)
    clf.fit(X_train_multilabel, y_train)
    y_pred = clf.predict(X_test_multilabel)
    print("Clf :",classifier)
    print("Accuracy :",metrics.accuracy_score(y_test,y_pred))
    print("Macro f1 score :",metrics.f1_score(y_test, y_pred, average = 'macro'))
    print("Micro f1 scoore :",metrics.f1_score(y_test, y_pred, average = 'micro'))
    print("Hamming loss :",metrics.hamming_loss(y_test,y_pred))
    print("-----------------------------------------------------------------------------------------------------------------------")

Clf : SGDClassifier()
Accuracy : 0.4383936080740118
Macro f1 score : 0.5561489479962561
Micro f1 scoore : 0.6143584768372544
Hamming loss : 0.06686291000841042
-----------------------------------------------------------------------------------------------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Clf : LogisticRegression()
Accuracy : 0.4401808242220353
Macro f1 score : 0.577787660815859
Micro f1 scoore : 0.6242589518615129
Hamming loss : 0.06663162321278385
-----------------------------------------------------------------------------------------------------------------------
Clf : LinearSVC()
Accuracy : 0.525336417157275
Macro f1 score : 0.6590744822210807
Micro f1 scoore : 0.688491213030433
Hamming loss : 0.061122792262405384
-----------------------------------------------------------------------------------------------------------------------
Clf : KNeighborsClassifier()
Accuracy : 0.3153910849453322
Macro f1 score : 0.4342046967117428
Micro f1 scoore : 0.47146877432489076
Hamming loss : 0.09279857022708159
-----------------------------------------------------------------------------------------------------------------------


In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier

mlp = MLPClassifier(activation='relu', alpha=0.05, hidden_layer_sizes=(10, 30, 10),learning_rate='adaptive',solver='adam')
gb = GradientBoostingClassifier(learning_rate=0.1, n_estimators=100)

for classifier in [mlp, gb]:
    clf = OneVsRestClassifier(classifier)
    clf.fit(X_train_multilabel, y_train)
    y_pred = clf.predict(X_test_multilabel)
    print("Clf :",classifier)
    print("Accuracy :",metrics.accuracy_score(y_test,y_pred))
    print("Macro f1 score :",metrics.f1_score(y_test, y_pred, average = 'macro'))
    print("Micro f1 scoore :",metrics.f1_score(y_test, y_pred, average = 'micro'))
    print("Hamming loss :",metrics.hamming_loss(y_test,y_pred))
    print("-----------------------------------------------------------------------------------------------------------------------")

/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Clf : MLPClassifier(alpha=0.05, hidden_layer_sizes=(10, 30, 10),
              learning_rate='adaptive')
Accuracy : 0.46604289318755254
Macro f1 score : 0.6069107515791214
Micro f1 scoore : 0.6324412889131622
Hamming loss : 0.08490328006728343
-----------------------------------------------------------------------------------------------------------------------
Clf : GradientBoostingClassifier()
Accuracy : 0.42378048780487804
Macro f1 score : 0.5678862242289168
Micro f1 scoore : 0.6034799656904791
Hamming loss : 0.068040370058873
-----------------------------------------------------------------------------------------------------------------------


In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree

dt=tree.DecisionTreeClassifier(criterion="gini",splitter="best",min_samples_split=2)
rf = RandomForestClassifier(n_estimators=1000,min_samples_split=10)

for classifier in [dt,rf]:
    clf = OneVsRestClassifier(classifier)
    clf.fit(X_train_multilabel, y_train)
    y_pred = clf.predict(X_test_multilabel)
    print("Clf :",classifier)
    print("Accuracy :",metrics.accuracy_score(y_test,y_pred))
    print("Macro f1 score :",metrics.f1_score(y_test, y_pred, average = 'macro'))
    print("Micro f1 scoore :",metrics.f1_score(y_test, y_pred, average = 'micro'))
    print("Hamming loss :",metrics.hamming_loss(y_test,y_pred))
    print("-----------------------------------------------------------------------------------------------------------------------")

Clf : DecisionTreeClassifier()
Accuracy : 0.4492220353238015
Macro f1 score : 0.6005310933996132
Micro f1 scoore : 0.6300925484209522
Hamming loss : 0.08151808242220353
-----------------------------------------------------------------------------------------------------------------------
Clf : RandomForestClassifier(min_samples_split=10, n_estimators=1000)
Accuracy : 0.5131412952060556
Macro f1 score : 0.6270478874662311
Micro f1 scoore : 0.6745795128536873
Hamming loss : 0.060817914213624896
-----------------------------------------------------------------------------------------------------------------------
